In [33]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.cross_validation import train_test_split
%matplotlib inline
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn import tree

In [84]:
df1 = pd.read_csv('lemon_training.csv')

In [85]:
df= df1.drop(["PRIMEUNIT","AUCGUART"], axis = 1)

In [86]:
df.info() 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51088 entries, 0 to 51087
Data columns (total 32 columns):
RefId                                51088 non-null int64
IsBadBuy                             51088 non-null int64
PurchDate                            51088 non-null object
Auction                              51088 non-null object
VehYear                              51088 non-null int64
VehicleAge                           51088 non-null int64
Make                                 51088 non-null object
Model                                51088 non-null object
Trim                                 49426 non-null object
SubModel                             51082 non-null object
Color                                51082 non-null object
Transmission                         51081 non-null object
WheelTypeID                          48819 non-null float64
WheelType                            48816 non-null object
VehOdo                               51088 non-null int64
Nationalit

In [87]:
df.dropna(axis=0, how='any', inplace=True)

In [88]:
df.info() 

<class 'pandas.core.frame.DataFrame'>
Int64Index: 47036 entries, 0 to 51087
Data columns (total 32 columns):
RefId                                47036 non-null int64
IsBadBuy                             47036 non-null int64
PurchDate                            47036 non-null object
Auction                              47036 non-null object
VehYear                              47036 non-null int64
VehicleAge                           47036 non-null int64
Make                                 47036 non-null object
Model                                47036 non-null object
Trim                                 47036 non-null object
SubModel                             47036 non-null object
Color                                47036 non-null object
Transmission                         47036 non-null object
WheelTypeID                          47036 non-null float64
WheelType                            47036 non-null object
VehOdo                               47036 non-null int64
Nationalit

In [89]:
df.head(3)

,RefId,IsBadBuy,PurchDate,Auction,VehYear,VehicleAge,Make,Model,Trim,SubModel,...,MMRCurrentAuctionAveragePrice,MMRCurrentAuctionCleanPrice,MMRCurrentRetailAveragePrice,MMRCurrentRetailCleanPrice,BYRNO,VNZIP1,VNST,VehBCost,IsOnlineSale,WarrantyCost
0,1,0,12/7/2009,ADESA,2006,3,MAZDA,MAZDA3,i,4D SEDAN I,...,7451.0,8552.0,11597.0,12409.0,21973,33619,FL,7100.0,0,1113
1,2,0,12/7/2009,ADESA,2004,5,DODGE,1500 RAM PICKUP 2WD,ST,QUAD CAB 4.7L SLT,...,7456.0,9222.0,11374.0,12791.0,19638,33619,FL,7600.0,0,1053
2,5,0,12/7/2009,ADESA,2005,4,FORD,FOCUS,ZX3,2D COUPE ZX3,...,3247.0,4384.0,6739.0,7911.0,19638,33619,FL,4000.0,0,1020


In [91]:
df["ModelNumeric"] = df["Model"].astype("category", ordered = True).cat.codes 
df["SubNumeric"] = df["SubModel"].astype("category", ordered = True).cat.codes
df['auctionscore'] = df['Auction'].astype("category", ordered = True).cat.codes
df['makescore'] = df['Make'].astype("category", ordered = True).cat.codes
df['transmissionscore'] = df['Transmission'].astype("category", ordered = True).cat.codes
df['vnstscore'] = df['VNST'].astype("category", ordered = True).cat.codes
df['nationalityscore'] = df['Nationality'].astype("category", ordered = True).cat.codes
df.head(3)

,RefId,IsBadBuy,PurchDate,Auction,VehYear,VehicleAge,Make,Model,Trim,SubModel,...,VehBCost,IsOnlineSale,WarrantyCost,ModelNumeric,SubNumeric,auctionscore,makescore,transmissionscore,vnstscore,nationalityscore
0,1,0,12/7/2009,ADESA,2006,3,MAZDA,MAZDA3,i,4D SEDAN I,...,7100.0,0,1113,503,199,0,16,0,5,2
1,2,0,12/7/2009,ADESA,2004,5,DODGE,1500 RAM PICKUP 2WD,ST,QUAD CAB 4.7L SLT,...,7600.0,0,1053,0,685,0,5,0,5,0
2,5,0,12/7/2009,ADESA,2005,4,FORD,FOCUS,ZX3,2D COUPE ZX3,...,4000.0,0,1020,317,49,0,6,1,5,0


In [93]:
dfnumeric= df.drop(["PurchDate","IsBadBuy","Auction","VNST","TopThreeAmericanName", "Make", "Model", "Nationality", "Trim", "SubModel", "Color", "Transmission","WheelType", "Size"], axis = 1)

In [94]:
dfnumeric.head(3)

,RefId,VehYear,VehicleAge,WheelTypeID,VehOdo,MMRAcquisitionAuctionAveragePrice,MMRAcquisitionAuctionCleanPrice,MMRAcquisitionRetailAveragePrice,MMRAcquisitonRetailCleanPrice,MMRCurrentAuctionAveragePrice,...,VehBCost,IsOnlineSale,WarrantyCost,ModelNumeric,SubNumeric,auctionscore,makescore,transmissionscore,vnstscore,nationalityscore
0,1,2006,3,1.0,89046,8155.0,9829.0,11636.0,13600.0,7451.0,...,7100.0,0,1113,503,199,0,16,0,5,2
1,2,2004,5,1.0,93593,6854.0,8383.0,10897.0,12572.0,7456.0,...,7600.0,0,1053,0,685,0,5,0,5,0
2,5,2005,4,2.0,69367,3913.0,5054.0,7723.0,8707.0,3247.0,...,4000.0,0,1020,317,49,0,6,1,5,0


In [95]:
y = df.IsBadBuy
X = dfnumeric

In [96]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [97]:
model = GradientBoostingClassifier().fit(X_train, y_train)

In [98]:
model.score(X_test, y_test) #this is the rsquared

0.90574668212859166

In [99]:
list(zip(X_test, model.feature_importances_))

[('RefId', 0.065189480710439063),
 ('VehYear', 0.0054453225118228608),
 ('VehicleAge', 0.10034887885870596),
 ('WheelTypeID', 0.0034372496517997577),
 ('VehOdo', 0.11395229221091582),
 ('MMRAcquisitionAuctionAveragePrice', 0.027147632218631458),
 ('MMRAcquisitionAuctionCleanPrice', 0.040185925225074776),
 ('MMRAcquisitionRetailAveragePrice', 0.032875584484162924),
 ('MMRAcquisitonRetailCleanPrice', 0.031735304022332649),
 ('MMRCurrentAuctionAveragePrice', 0.018540050012387531),
 ('MMRCurrentAuctionCleanPrice', 0.023999251161394494),
 ('MMRCurrentRetailAveragePrice', 0.045017970437435037),
 ('MMRCurrentRetailCleanPrice', 0.016239980410403358),
 ('BYRNO', 0.050951471391159589),
 ('VNZIP1', 0.04309400325639156),
 ('VehBCost', 0.12582817720945971),
 ('IsOnlineSale', 0.00119514722709955),
 ('WarrantyCost', 0.046169109687269227),
 ('ModelNumeric', 0.06497921002895303),
 ('SubNumeric', 0.059850725372368026),
 ('auctionscore', 0.018209720229411273),
 ('makescore', 0.032795821939797298),
 ('tra

In [73]:
cv = KFold(n_splits=10)
for estimators in range(100, 1000, 100):
    for depth in range(1,23):
        model = GradientBoostingClassifier(n_estimators=estimators, max_depth=depth)
        scores = []
        for train_i, test_i in cv.split(X):
            Xr, yr, Xt, yt = X.loc[train_i], y.loc[train_i], X.loc[test_i], y.loc[test_i]
            model.fit(Xr, yr)
            scores.append(model.score(Xt, yt))
        print('depth:' , depth, 'estimators:', estimators, sum(scores)/len(scores),'min:', min(scores), 'max:', max(scores))

ValueError: Input contains NaN, infinity or a value too large for dtype('float32').